In [1]:
import cv2
import numpy as np
import face_recognition

In [2]:
# imgelon_bgr = face_recognition.load_image_file('dataset\\genuine\\WIN_20240307_21_52_24_Pro.jpg')
# imgelon_rgb = cv2.cvtColor(imgelon_bgr,cv2.COLOR_BGR2RGB)
# cv2.imshow('bgr', imgelon_bgr)
# cv2.imshow('rgb', imgelon_rgb)
# cv2.waitKey(0)

In [3]:
# imgelon =face_recognition.load_image_file('dataset\\genuine\\WIN_20240307_21_52_24_Pro.jpg')
# imgelon = cv2.cvtColor(imgelon,cv2.COLOR_BGR2RGB)
# #----------Finding face Location for drawing bounding boxes-------
# face = face_recognition.face_locations(imgelon_rgb)[0]
# copy = imgelon.copy()
# #-------------------Drawing the Rectangle-------------------------
# cv2.rectangle(copy, (face[3], face[0]),(face[1], face[2]), (255,0,255), 2)
# cv2.imshow('copy', copy)
# cv2.imshow('elon',imgelon)
# cv2.waitKey(0)

In [4]:
# train_elon_encodings = face_recognition.face_encodings(imgelon)[0]

In [5]:
# # lets test an image
# test = face_recognition.load_image_file('dataset\\genuine\\aman gupta.png')
# test = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)
# test_encode = face_recognition.face_encodings(test)[0]
# print(face_recognition.compare_faces([train_elon_encodings],test_encode))

In [6]:
import cv2
import face_recognition
import os
import numpy as np
from datetime import datetime
import pickle

In [7]:
dataset_path = 'dataset/students'

In [8]:
images = []
classNames = []
# Dictionary to store encodings of each person
encodeDict = {}
mylist = os.listdir(dataset_path)
for cl in mylist:
    # Read the image
    curImg = cv2.imread(os.path.join(dataset_path, cl))
    if curImg is not None:
        # Append the image to the list
        images.append(curImg)
        # Get the class name (person's name) from the file name
        className = os.path.splitext(cl)[0]
        classNames.append(className)
        
        # Convert image to RGB (face_recognition library requires RGB format)
        rgb_img = cv2.cvtColor(curImg, cv2.COLOR_BGR2RGB)
        # Find face encodings
        face_encodings = face_recognition.face_encodings(rgb_img)
        if len(face_encodings) > 0:
            # If the person already exists in the dictionary, append the encoding
            if className in encodeDict:
                encodeDict[className].append(face_encodings[0])
            # Otherwise, create a new entry in the dictionary
            else:
                encodeDict[className] = [face_encodings[0]]
        else:
            print(f"No face found in the image: {cl}")



In [9]:
# Loop through each subdirectory (person) in the dataset directory
for person_dir in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_dir)
    
    # Check if it's a directory
    if os.path.isdir(person_path):
        # Get the list of image files in the subdirectory
        image_files = [f for f in os.listdir(person_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        
        # Loop through each image file
        for img_file in image_files:
            img_path = os.path.join(person_path, img_file)
            # Read the image
            curImg = cv2.imread(img_path)
            if curImg is not None:
                # Append the image to the list
                images.append(curImg)
                # Append the class name (person's name) to the classNames list
                classNames.append(person_dir)
                
                # Convert image to RGB (face_recognition library requires RGB format)
                rgb_img = cv2.cvtColor(curImg, cv2.COLOR_BGR2RGB)
                # Find face encodings
                face_encodings = face_recognition.face_encodings(rgb_img)
                if len(face_encodings) > 0:
                    # If the person already exists in the dictionary, append the encoding
                    if person_dir in encodeDict:
                        encodeDict[person_dir].append(face_encodings[0])
                    # Otherwise, create a new entry in the dictionary
                    else:
                        encodeDict[person_dir] = [face_encodings[0]]
                else:
                    print(f"No face found in the image: {img_path}")
            else:
                print(f"Unable to read image: {img_path}")

In [10]:
# # Function to encode faces in the images
# def findEncodings(images):
#     encodeList = []
#     for img in images:
#         # Convert image to RGB (face_recognition library requires RGB format)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         # Find face encodings
#         face_encodings = face_recognition.face_encodings(img)
#         if len(face_encodings) > 0:
#             encodeList.append(face_encodings[0])
#         else:
#             print("No face found in the image")
#     return encodeList

In [11]:
# # Encode faces in the loaded images
# encoded_face_train = findEncodings(images)

In [12]:
# def markAttendance(name):
#     with open('Attendance.csv','r+') as f:
#         myDataList = f.readlines()
#         nameList = []
#         for line in myDataList:
#             entry = line.split(',')
#             nameList.append(entry[0])
#         if name not in nameList:
#             now = datetime.now()
#             time = now.strftime('%I:%M:%S:%p')
#             date = now.strftime('%d-%B-%Y')
#             f.writelines(f'n{name}, {time}, {date}')

In [13]:
# Set threshold for face matching
face_match_threshold = 0.4

# Capture images from webcam
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0,0), None, 0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    faces_in_frame = face_recognition.face_locations(imgS)
    encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)
    for encode_face, faceloc in zip(encoded_faces, faces_in_frame):
        # Initialize matchIndex and min_distance
        matchIndex = None
        min_distance = float('inf')
        
        # Compare the face encoding with the stored encodings
        for className, encoding_list in encodeDict.items():
            for encoding in encoding_list:
                distance = face_recognition.face_distance([encoding], encode_face)
                if distance < min_distance:
                    min_distance = distance
                    matchIndex = className
        
        if matchIndex is not None and min_distance <= face_match_threshold:
            name = matchIndex.upper().lower()
        else:
            name = "unknown"
        
        y1, x2, y2, x1 = faceloc
        # since we scaled down by 4 times
        y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.rectangle(img, (x1,y2-35), (x2,y2), (0,255,0), cv2.FILLED)
        cv2.putText(img, name, (x1+6,y2-5), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2)
        
    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
# Release the webcam
cap.release()
cv2.destroyAllWindows()